# Concolic Fuzzing

We have previously seen how one can use dynamic taints to produce more intelligent test cases than simply looking for program crashes. We have also seen how one can use the taints to update the grammar, and hence focus more on the dangerous methods. 

While taints are helpful, uninterpreted strings is only one of the attack vectors. Can we say anything more about the properties of variables at any point in the execution? For example, can we say for sure that a function will always receive the buffers with the correct length? Concolic execution offers a solution. With concolic execution, one can collect the constraints that an execution path encounters, and use it to answer questions about the program behavior at any point we prefer. We can even use concolic execution to enhance fuzzing.

In this chapter, we explore in depth how to execute a Python function concolically, and how concolic execution can be used to enhance fuzzing.

**Prerequisites**

* You should have read the [chapter on coverage](Coverage.ipynb).
* You should have read the [chapter on information flow](InformationFlow.ipynb).
* A familiarity with the basic idea of [SMT solvers](https://en.wikipedia.org/wiki/Satisfiability_modulo_theories) would be useful.

We first setup our infrastructure so that we can make use of previously defined functions.

In [ ]:
import fuzzingbook_utils

## Tracking Constraints

In the chapter on [information flow](InformationFlow), we have seen how dynamic taints can be used to direct fuzzing by indicating which part of input reached interesting places. However, dynamic taint tracking is limited in the information that it can propagate. For example, we might want to explore what happens when certain properties of the input changes.

For example, say we have a function `factorial()` that returns the *factorial value* of its input.

In [ ]:
def factorial(n):
    if n < 0:
        return None
    if n == 0:
        return 1
    if n == 1:
        return 1
    v = 1
    while n != 0:
        v = v * n
        n = n - 1
    return v

We exercise the function with a value of `5`.

In [ ]:
factorial(5)

Is this sufficient to explore all the features of the function? How do we know? One way to verify that we have explored all features is to look at the coverage obtained. First we need to extend the `Coverage` class from the [chapter on coverage](Coverage.ipynb) to provide us with coverage arcs.

In [ ]:
from Coverage import Coverage

In [ ]:
import inspect

In [ ]:
class ArcCoverage(Coverage):
    def traceit(self, frame, event, args):
        if event != 'return':
            f = inspect.getframeinfo(frame)
            self._trace.append((f.function, f.lineno))
        return self.traceit

    def arcs(self):
        t = [i for f, i in self._trace]
        return list(zip(t, t[1:]))

Next, we use the `Tracer` to obtain the coverage arcs.

In [ ]:
with ArcCoverage() as cov:
    factorial(5)

We can now use the coverage arcs to visualize the coverage obtained.

In [ ]:
from ControlFlow import PyCFG, CFGNode, to_graph, gen_cfg

In [ ]:
from graphviz import Source, Graph

In [ ]:
Source(to_graph(gen_cfg(inspect.getsource(factorial)), arcs=cov.arcs()))

We see that the path `[1, 2, 4, 6, 8, 9, 10, 11, 12]` is covered but sub-paths such as  `[2, 3]`,  `[4, 5]` and `[6, 7]` are unexplored. What we need is the ability to generate inputs such that the `True` branch is taken at `2`. How do we do that?

## Concolic Execution

One way is to look at the execution path being taken, and collect the conditional constraints that the path encounters. Then we can try to produce inputs that lead us to taking the non-traversed path.

First, let us step through the function.

In [ ]:
lines = [i[1] for i in cov._trace if i[0] == 'factorial']
src = {i+1:s for i,s in enumerate(inspect.getsource(factorial).split('\n'))}

* The line (1) is simply the entry point of the function. We know that the input is `n`, which is an integer.

In [ ]:
src[1]

* The line (2) is a predicate `n < 0`. Since the next line taken is line (4), we know that at this point in the execution path, the predicate was `true`.

In [ ]:
src[2], src[3], src[4]

We notice that this is one of the predicates where the `true` branch was not taken. How do we generate a value that takes the `true` branch here? One way is to use symbolic variables to represent the input, encode the constraint, and use an *SMT Solver* to solve the negation of the constraint.

A symbolic variable can be thought of as a sort of placeholder for the real variable, sort of like the `x` in solving for `x` in Algebra. We identify what constraints the variable is supposed to obey, and finally produce a value that obeys all constraints imposed.

We first import an SMT solver, *Z3*.

In [ ]:
import z3

Encoding the constraint requires declaring a corresponding symbolic variable to the input `n`.

In [ ]:
zn = z3.Int('n')

We can now encode the constraint as follows.

In [ ]:
zn < 0

We use *Z3* to verify that the input `5` would actually result in negation of this constraint.

In [ ]:
z3.solve(z3.And(z3.Not(zn < 0), zn == 5))

Similarly, we can ask *Z3* for an input that satisfies the constraint.

In [ ]:
z3.solve(zn < 0)

Let us try using that with our coverage.

In [ ]:
with cov as cov:
    factorial(-1)

In [ ]:
Source(to_graph(gen_cfg(inspect.getsource(factorial)), arcs=cov.arcs()))

Ok, so we have managed to cover a little more of the graph. Let us continue with our original input of `factorial(5)`:
* In line (4) we encounter a new predicate `n == 0`, for which we again took the false branch.

In [ ]:
src[4]

The predicates required to follow the path until this point is as follows.

In [ ]:
predicates = [z3.Not(zn < 0), z3.Not(zn == 0)]

* If we continue to line (6), we encounter another predicate, for which again, we took the `false` branch

In [ ]:
src[6]

The predicates encountered so far is as follows

In [ ]:
predicates = [z3.Not(zn < 0), z3.Not(zn == 0), z3.Not(zn == 1)]

To take the branch at (6), we essentially have to obey the predicates until that point, but invert the last predicate.

In [ ]:
last = len(predicates) - 1
z3.solve(predicates[0:-1] + [z3.Not(predicates[-1])])

What we are doing here is tracing the execution corresponding to a particular input `factorial(5)`, using concrete values, and along with it, keeping *symbolic shadow variables* that enable us to capture the constraints. This particular method of execution where one combines concrete and symbolic execution is called *Concolic Execution*

### Concolic Wrappers

In [ ]:
from InformationFlow import *

We first define a class for taking care of wrapping arguments and providing context. It accepts the function to be traced concolically.

In [ ]:
class ConcolicTracer:
    def __init__(self, info=None):
        self.info = info if info is not None else ({}, [])
        self.decls, self.path = self.info

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, tb):
        return

    def __getitem__(self, fn):
        self.fn = fn
        self.fn_args = {
            i: None for i in list(
                inspect.signature(fn).parameters)}
        return self

    def __call__(self, *args):
        self.result = self.fn(*self.prepare_args(args))
        return self.result

For ease of use, we use an [SMT solver](https://en.wikipedia.org/wiki/Satisfiability_modulo_theories) called [z3](http://theory.stanford.edu/~nikolaj/programmingz3.html). *Z3* allows us to both encode the constraints encountered in _symbolic variables_ and also to solve these constraints at the end.   A *symbolic variable* can be thought of as a sort of placeholder for the real variable, sort of like the `x` in solving for `x` in Algebra. We identify what conditions the variable is supposed to obey, and finally produce a value that obeys all.

In [ ]:
import z3

In [ ]:
z3.get_version()  # should be 4.8.5

We also need to make sure that the latest `z3str3` solver is used, and the timeout is 30 seconds.

In [ ]:
z3.set_option('smt.string_solver', 'z3str3')
z3.set_option('timeout', 30 * 1000)  # milliseconds

We can declare symbolic variables as below

In [ ]:
a, b, c = z3.Int('a'), z3.Int('b'), z3.Int('c')

With these, we can now capture constraints such as below

In [ ]:
eqs = z3.And(a == b, a != c, b != c)
eqs

The _z3_ SMT solver can also be used to solve such a system of equations as follows

In [ ]:
z3.solve(eqs)

The SMT solver will respond with _no solutions_ if no solutions could be found.

In [ ]:
z3.solve(z3.And(a == b, a == c, b != c))

We use these symbolic variables to capture constraints in the concolic wrapper classes we define below.

#### zbool

First, we define the `zbool` class which is used to track the predicates encountered. It is a wrapper class that contains both symbolic (`z`) as well as concrete (`v`) values. The concrete value is used to determine which path to take, and the symbolic value is used to collect the predicates encountered.

In [ ]:
class zbool:
    @classmethod
    def create(cls, info, zn, v):
        zv = zbool(info, z3.Bool(zn), v)
        info[0][zn] = 'Bool'
        return z

    def __init__(self, info, z, v=None):
        self.info, self.z, self.v = info, z, v

    def __not__(self):
        return zbool(self.info, z3.Not(self.z), not self.v)

    def __bool__(self):
        r, pred = (True, self.z) if self.v else (False, z3.Not(self.z))
        self.info[1].append(pred)
        return r

The `zbool` class can be used to keep track of boolean values and conditions encountered during the execution as below.

In [ ]:
with ConcolicTracer() as _:
    za, zb, zc = z3.Ints('a b c')
    ca, cb, cc = 1, 1, 2
    if zbool(_.info, za == zb, ca == cb):
        if zbool(_.info, zb == zc, cb == cc):
            print('equilateral')
        else:
            print('isosceles')

We can retrieve the predicates encountered as follows.

In [ ]:
z3.And(_.path).sexpr()

#### zint

Next, we define a symbolic wrapper `zint` for `int`.

The class keeps track of the int variables used and the predicates encountered in `info`. Finally, it also keeps the concrete value used so that it can be used to determine the path to take.

In [ ]:
class zint(int):
    def __new__(cls, info, zn, v, *args, **kw):
        return int.__new__(cls, v, *args, **kw)

    @classmethod
    def create(cls, info, zn, v=None):
        zv = zint(info, z3.Int(zn), v)
        info[0][zn] = 'Int'
        return zv

    def __init__(self, info, z, v=None):
        self.z, self.v = z, v
        self.info = info

    def zv(self, o):
        return (o.z, o.v) if isinstance(o, zint) else (z3.IntVal(o), o)

    def __ne__(self, other):
        z, v = self.zv(other)
        return zbool(self.info, self.z != z, self.v != v)

    def __eq__(self, other):
        z, v = self.zv(other)
        return zbool(self.info, self.z == z, self.v == v)

    def __lt__(self, other):
        z, v = self.zv(other)
        return zbool(self.info, self.z < z, self.v < v)

    def __gt__(self, other):
        z, v = self.zv(other)
        return zbool(self.info, self.z > z, self.v > v)

    def __req__(self, other):
        return self.__eq__(other)

Using these proxies is similar to our taint tracking proxies.

While using the proxy classes, we often will have to generate new symbolic variables, with names that have not been used before. For this, we define `fresh_name()` that always generates unique integers for names.

In [ ]:
COUNTER = 0

In [ ]:
def fresh_name():
    global COUNTER
    COUNTER += 1
    return COUNTER

 We wrap the arguments with appropriate classes using `ConcolicTracer`.

In [ ]:
class ConcolicTracer(ConcolicTracer):
    def prepare_args(self, args):
        my_args = []
        for name, arg in zip(self.fn_args, args):
            t = type(arg).__name__
            zwrap = globals()['z' + t]
            vname = "%s_%s_%s_%s" % (self.fn.__name__, name, t, fresh_name())
            my_args.append(zwrap.create(self.info, vname, arg))
            self.fn_args[name] = vname
        return my_args

In [ ]:
class ConcolicTracer(ConcolicTracer):
    def pred(self, show_decl=False, simplify=False):
        r = []
        if show_decl:
            for decl in self.decls:
                v = self.decls[decl]
                v = '(_ BitVec 8)' if v == 'BitVec' else v
                r.append("(declare-const %s %s)" % (decl, v))
        if self.path:
            if show_decl:
                if simplify:
                    return '\n'.join([*r, "(assert %s)" %
                                      z3.simplify(z3.And(self.path)).sexpr()])
                else:
                    return '\n'.join([*r, "(assert %s)" %
                                      z3.And(self.path).sexpr()])
            else:
                return z3.simplify(z3.And(self.path)).sexpr()
        else:
            return ''

Next, we invoke the function under `ConcolicTracer`

In [ ]:
with ConcolicTracer() as _:
    result = _[abs_value](1)

The predicates encountered are captured in the `path`

In [ ]:
_.pred()

If we want to solve the path conditions again, we simply need to declare the symbolic variables

In [ ]:
for decl in _.decls:
    v = "z3.%s('%s')" % (_.decls[decl], decl)
    print(v)
    exec(v)

And call `smt.solve()` on the `path`.

In [ ]:
z3.solve(_.path)

If we want to explore other paths, we simply need to invert the corresponding predicate.

In [ ]:
z3.solve(z3.Not(_.path[0]))

We can verify that the path taken is as expected.

In [ ]:
with ArcCoverage() as cov:
    abs_value(-1)

In [ ]:
Source(to_graph(gen_cfg(inspect.getsource(abs_value)), arcs=cov.arcs()))

Here is another function that gives you the nearest ten's multiplier

In [ ]:
def round10(r):
    while r % 10 != 0:
        r += 1
    return r

For dealing with this, we need a few more helper methods.

In [ ]:
class zint(zint):
    def __add__(self, other):
        z, v = self.zv(other)
        return zint(self.info, z + self.z, self.v + v)  # z3 bug

    def __radd__(self, other):
        return self.__add__(other)

    def __mod__(self, other):
        z, v = self.zv(other)
        return zint(self.info, self.z % z, self.v % v)

    def __neg__(self):
        return zint(self.info, -self.z, -self.v)

As before, we execute the function under the `ConcolicTracer` context.

In [ ]:
with ConcolicTracer() as _:
    r = _[round10](1)

We verify that it has worked as expected.

In [ ]:
_.info

The predicates encountered can be seen in the path.

In [ ]:
for decl in _.decls:
    v = "(declare-const %s %s)" % (decl, _.decls[decl])
    print(v)
print(_.pred())

Does `zint` work across functions? Say we have a function `max_value()` as below.

In [ ]:
def abs_max(a, b):
    a1 = abs_value(a)
    b1 = abs_value(b)
    if a1 > b1:
        c = a1
    else:
        c = b1
    return c

Using the `Concolic()` context on `abs_max()`.

In [ ]:
with ConcolicTracer() as _:
    _[abs_max](2, 1)

As expected, we have the predicates across functions.

In [ ]:
_.info

Using negative numbers makes it behave as expected.

In [ ]:
with ConcolicTracer() as _:
    _[abs_max](-2, -1)

In [ ]:
_.info

We define `zeval()` to make the process of evaluation simpler.

In [ ]:
def zeval_direct(path, cc):
    s = cc.pred(True, True)
    with open('_.smt', 'w+') as f:
        f.write(s)
        f.write("\n(check-sat)\n(get-model)")

In [ ]:
def zeval_py(path, cc):
    for decl in cc.decls:
        v = "z3.%s('%s')" % (cc.decls[decl], decl)
        exec(v)
    s = z3.Solver()
    s.add(z3.And(path))
    if s.check() != z3.sat:
        return 'No solution'
    m = s.model()
    solutions = {d.name(): m[d] for d in m.decls()}
    my_args = {k: solutions.get(cc.fn_args[k], None) for k in cc.fn_args}
    return my_args

In [ ]:
def zeval(cc, direct=False):
    if direct:
        return zeval_direct(cc.path, cc)
    else:
        return zeval_py(cc.path, cc)

In [ ]:
zeval(_)

Sometimes the Python version can be a little buggy. In that case, we can use z3 directly for evaluating the symbolic expressions.

In [ ]:
zeval(_, True)

In [ ]:
!cat _.smt

In [ ]:
!z3 _.smt

#### zstr

Here, we define the wrappers for strings `zstr`.

First, we define `zord` that given a symbolic one character long string, obtains the `ord()` for that. It returns two values. The first one is the variable that corresponds to `ord()`, and second is the predicate that links the variable to the passed in single character string.

In [ ]:
def zord(info, c):
    bn = "bitvec_%d" % fresh_name()
    v = z3.BitVec(bn, 8)
    info[0][bn] = 'BitVec'
    z = (z3.Unit(v) == c)
    info[1].append(z)
    return v

We use it as follows

In [ ]:
zc = z3.String('arg_%d' % fresh_name())

In [ ]:
with ConcolicTracer() as _:
    zi = zord(_.info, zc)

The symbolic bitvector is in `zi`. It is linked to the passed in argument in `info`

In [ ]:
_.info

We can specify what the result of `ord()` should be, and call `z3.solve()` to provide us with a solution that will provide the required result.

In [ ]:
z3.solve(_.path + [zi == 65])

Similarly, we can convert the ASCII value back to a single character string using `zchr()`

In [ ]:
def zchr(info, i):
    sn = 'string_%d' % fresh_name()
    s = z3.String(sn)
    info[0][sn] = 'String'
    z = z3.And([s == z3.Unit(i), z3.Length(s) == 1])
    info[1].append(z)
    return s

For using it, we first define a bitvector that is 8 bits long.

In [ ]:
i = z3.BitVec('bv_%d' % fresh_name(), 8)

We can now retrieve the `chr()` representation as below.

In [ ]:
with ConcolicTracer() as _:
    zc = zchr(_.info, i)

In [ ]:
_.info

As before, we can specify what the end result of calling `chr()` should be to get the original argument.

In [ ]:
z3.solve(_.path + [zc == z3.StringVal('a')])

With these, we can now define our `zstr` class.

In [ ]:
class zstr(str):
    def __new__(cls, info, zn, v):
        return str.__new__(cls, v)

    @classmethod
    def create(cls, info, zn, v=None):
        z = z3.String(zn)
        zv = zstr(info, z, v)
        info[0][zn] = 'String'
        return zv

    def __init__(self, info, z, v=None):
        self.info, self.z, self.v = info, z, v
        self._len = zint(info, z3.Length(z), len(v))

    def zv(self, o):
        return (o.z, o.v) if isinstance(o, zstr) else (z3.StringVal(o), o)

    def __eq__(self, other):
        z, v = self.zv(other)
        return zbool(self.info, self.z == z, self.v == v)

    def __req__(self, other):
        return self.__eq__(other)

The `zstr` class is used as follows.

In [ ]:
def tstr1(s):
    if s == 'h':
        return True
    else:
        return False

In [ ]:
with ConcolicTracer() as _:
    r = _[tstr1]('h')

In [ ]:
_.info

It works even if we have more than one character.

In [ ]:
def tstr1(s):
    if s == 'hello':
        return True
    else:
        return False

In [ ]:
with ConcolicTracer() as _:
    r = _[tstr1]('hello')

In [ ]:
zeval(_)

What if we need to concatenate two strings? We need additional helpers to accomplish that.

In [ ]:
class zstr(zstr):
    def __add__(self, other):
        z, v = self.zv(other)
        return zstr(self.info, self.z + z, self.v + v)

    def __radd__(self, other):
        return self.__add__(other)

Here is how it can be used. First, we create the wrapped arguments

In [ ]:
with ConcolicTracer() as _:
    v1, v2 = [zstr.create(_.info, 'arg_%d' % fresh_name(), s)
              for s in ['hello', 'world']]
    if (v1 + ' ' + v2) == 'hello world':
        print('hello world')

The addition of symbolic variables is preserved in `info`

In [ ]:
_.info

Similarly, accessing substrings also require extra help.

In [ ]:
class zstr(zstr):
    def __getitem__(self, idx):
        if isinstance(idx, slice):
            start, stop, step = idx.indices(len(self.v))
            assert step == 1  # for now
            assert stop >= start  # for now
            rz = z3.SubString(self.z, start, stop - start)
            rv = self.v[idx]
        elif isinstance(idx, int):
            rz = z3.SubString(self.z, idx, 1)
            rv = self.v[idx]
        else:
            assert False  # for now
        return zstr(self.info, rz, rv)

    def __iter__(self):
        return zstr_iterator(self.info, self)

We define the iterator as follows.

In [ ]:
class zstr_iterator():
    def __init__(self, info, zstr):
        self.info = info
        self._zstr = zstr
        self._str_idx = 0
        self._str_max = zstr._len  # intz is not an _int_

    def __next__(self):
        if self._str_idx == self._str_max:  # intz#eq
            raise StopIteration
        c = self._zstr[self._str_idx]
        self._str_idx += 1
        return c

    def __len__(self):
        return self._len

Here is how it can be used.

In [ ]:
def tstr2(s):
    if s[0] == 'h' and s[1] == 'e' and s[3] == 'l':
        return True
    else:
        return False

In [ ]:
with ConcolicTracer() as _:
    r = _[tstr2]('hello')

In [ ]:
_.info

In [ ]:
zeval(_)

A major complication is supporting upper and lower methods.

In [ ]:
class zstr(zstr):
    def upper(self):
        empty = ''
        ne = 'empty_%d' % fresh_name()
        result = zstr.create(self.info, ne, empty)
        self.info[1].append(z3.StringVal(empty) == result.z)
        cdiff = (ord('a') - ord('A'))
        for i in self:
            oz = zord(self.info, i.z)
            uz = zchr(self.info, oz - cdiff)
            rz = z3.And([oz >= ord('a'), oz <= ord('z')])
            ov = ord(i.v)
            uv = chr(ov - cdiff)
            rv = ov >= ord('a') and ov <= ord('z')
            if zbool(self.info, rz, rv):
                i = zstr(self.info, uz, uv)
            else:
                i = zstr(self.info, i.z, i.v)
            result += i
        return result

In [ ]:
for name, fn in inspect.getmembers(str, callable):
    if name in ['__format__', 'format_map', 'format',
                '__mul__', '__rmul__', 'center', 'zfill', 'decode', 'encode']:
        setattr(zstr, name, make_str_abort_wrapper(fn))

Here is how it is used.

In [ ]:
def tstr3(s):
    if s.upper() == 'H':
        return True
    else:
        return False

In [ ]:
with ConcolicTracer() as _:
    r = _[tstr3]('h')

In [ ]:
print(_.pred(True, True))

The definition of `BitVec` variables are slightly different. Hence, we update `zeval()`

In [ ]:
def zeval_py(path, cc):
    for decl in cc.decls:
        if cc.decls[decl] == 'BitVec':
            v = "z3.%s('%s', 8)" % (cc.decls[decl], decl)
        else:
            v = "z3.%s('%s')" % (cc.decls[decl], decl)
        exec(v)
    s = z3.Solver()
    s.add(z3.And(path))
    if s.check() != z3.sat:
        return 'No solution'
    m = s.model()
    solutions = {d.name(): m[d] for d in m.decls()}
    my_args = {k: solutions.get(cc.fn_args[k], None) for k in cc.fn_args}
    return my_args

In [ ]:
zeval(_)

Here is a larger example using `upper()`

In [ ]:
def tstr4(s):
    if s.upper() == 'HELLO WORLD':
        return True
    else:
        return False

In [ ]:
with ConcolicTracer() as _:
    r = _[tstr4]('hello world')

In [ ]:
print(_.pred(True, True))

In [ ]:
zeval(_, direct=True)

In [ ]:
!z3 _.smt

We define `startswith()`.

In [ ]:
class zstr(zstr):
    def startswith(self, other, beg=0, end=None):
        assert end is None  # for now
        assert isinstance(beg, int)  # for now
        zb = z3.IntVal(beg)

        others = other if isinstance(other, tuple) else (other, )

        last = False
        for o in others:
            z, v = self.zv(o)
            r = z3.IndexOf(self.z, z, zb)
            last = zbool(self.info, r == zb, self.v.startswith(v))
            if last:
                return last
        return last

We also define `find()`

In [ ]:
class zint(zint):
    def __ge__(self, other):
        if self > other or self == other:
            return True
        return False

In [ ]:
class zstr(zstr):
    def find(self, other, beg=0, end=None):
        assert end is None  # for now
        assert isinstance(beg, int)  # for now
        zb = z3.IntVal(beg)
        z, v = self.zv(other)
        zi = z3.IndexOf(self.z, z, zb)
        vi = self.v.find(v, beg, end)
        return zint(self.info, zi, vi)

Using it.

In [ ]:
with ConcolicTracer() as _:
    v1 = zstr.create(_.info, 'arg_%d' % fresh_name(), 'hello')
    if v1.find('e') >= 0:
        print('found')

In [ ]:
_.info

In [ ]:
z3.solve(_.path)

We are now ready to target our `DB` class. Hash functions are difficult to handle directly (because it relies on internal C functions). Hence we modify `table` slightly.

In [ ]:
class ConcolicDB(DB):
    def table(self, t_name):
        for k, v in self.db:
            if t_name == k:
                return v
        raise SQLException('Table (%s) was not found' % repr(t_name))

To make it easy, we define a single function `db_select` that directly invokes `db.sql`.

In [ ]:
def db_select(s):
    try:
        my_db = ConcolicDB()
        my_db.db = [(k, v) for (k, v) in db.db.items()]
        r = my_db.sql(s)
        return r
    except:
        traceback.print_exc()
        return None

In [ ]:
with ConcolicTracer() as _:
    _[db_select]('select kind from inventory')

In [ ]:
print(_.pred(True, True))

In [ ]:
zeval(_, direct=True)

In [ ]:
!cat _.smt

In [ ]:
!z3 _.smt

__Note:__ If you get _unknown_ here, it means that the Z3 SMT solver gave up before it reached an answer. That is perfectly fine.

### Fuzzing with constraints

The concolic framework can be used directly in fuzzing. First, we extend our `TaintedGrammarFuzzer` with a few helper methods.

In [ ]:
class CGrammarFuzzer(TaintedGrammarFuzzer):
    def tree_to_string(self, tree):
        symbol, children, *_ = tree
        e = ostr('')
        if children:
            return e.join([self.tree_to_string(c) for c in children])
        else:
            return e if symbol in self.grammar else symbol

    def prune_tree(self, tree, tokens):
        name, children = tree
        children = self.coalesce(children)
        if name in tokens:
            return (name, [(self.tree_to_string(tree), [])])
        else:
            return (name, [self.prune_tree(c, tokens) for c in children])

    def coalesce(self, children):
        last = ostr('')
        new_lst = []
        for cn, cc in children:
            if cn not in self.grammar:
                last += cn
            else:
                if last:
                    new_lst.append((last, []))
                    last = ostr('')
                new_lst.append((cn, cc))
        if last:
            new_lst.append((last, []))
        return new_lst

We can now use the fuzzer to produce inputs for our DB.

In [ ]:
tgf = CGrammarFuzzer(INVENTORY_GRAMMAR)
qtree = tgf.fuzz_tree()
query = tgf.tree_to_string(qtree)

with ExpectError():
    print(repr(query))
    with ConcolicTracer() as _:
        res = _[db_select](str(query))
    print(repr(res))
    print(_.pred(True, True))

Our fuzzer returns with an exception. It is unable to find the specified table. Let us examine the predicates it encountered.

In [ ]:
for i, p in enumerate(_.path):
    print(i, p)

Note that we can obtain constraints that are not present in the grammar from using the `ConcolicTracer`. In particular, see how we are able to obtain the condition that the table needs to be `inventory` (8) for the fuzzing to succeed.

How do we lift these to the grammar? and in particular how do we do it automatically? One option we have is to simply switch the last predicate obtained. In our case, the last predicate is (6). Can we simply invert the predicate and solve it again?

In [ ]:
new_path = _.path[0:-1] + [z3.Not(_.path[-1])]

In [ ]:
new_ = ConcolicTracer((_.decls, new_path))
new_.fn = _.fn
new_.fn_args = _.fn_args

In [ ]:
zeval(new_, direct=True)

In [ ]:
!cat _.smt

In [ ]:
!z3 _.smt

Indeed, this will not work as the string lengths being compared to are different.

In [ ]:
print(_.path[-1])
z3.solve(z3.Not(_.path[-1]))

A better idea is to investigate what _string_ comparisons are being made, and associate that with the corresponding nodes in the grammar. Let us examine our derivation tree (pruned to avoid recursive structures, and to focus on important parts).

In [ ]:
dt = tgf.prune_tree(qtree, ['<assignments>', '<names>',
                            '<literals>', '<exprs>', '<bexpr>', '<table>'])
dt

Can we identify which part of the input was supplied by which part of the grammar? We define `span()` that can recover this information from the derivation tree.

In [ ]:
def span(node, g, node_start=0):
    hm = {}
    k, cs = node
    end_i = node_start
    new_cs = []
    for c in cs:
        chm, (ck, child_start, child_end, gcs) = span(c, g, end_i)
        new_cs.append((ck, child_start, child_end, gcs))
        end_i = child_end
        hm.update(chm)
    node_end = end_i if cs else node_start + len(k)
    if k in g and k != START_SYMBOL:
        hm[k] = (node_start, node_end - node_start)
    return hm, (k, node_start, node_end, new_cs)

We use it as follows

In [ ]:
span_hm, _n = span(dt, INVENTORY_GRAMMAR)

In [ ]:
span_hm

We can check if we got the right values as follows.

In [ ]:
print("query:", query)
for k in span_hm:
    start, l = span_hm[k]
    print(k, query[start:start + l])

Next, we need to obtain all the comparisons made in each predicate. For that, we define two helper functions. The first is `unwrap_substrings()` that translates multiple calls to `z3.SubString` and returns the start, and length of the given z3 string expression.

In [ ]:
def unwrap_substrings(s):
    assert s.decl().name() == 'str.substr'
    cs, frm, l = s.children()
    if cs.decl().name() == 'str.substr':
        newfrm, _l = unwrap_substrings(cs)
        return (frm.as_long() + newfrm, l.as_long())
    else:
        return (frm.as_long(), l.as_long())

Next, we define `traverse_z3()` that traverses a given z3 string expression, and collects all direct string comparisons to a substring of the original argument.

In [ ]:
def traverse_z3(p, hm):
    n = p.decl().name()
    if n == 'not':
        return traverse_z3(p.children()[0], hm)
    elif n == '=':
        i, j = p.children()
        if isinstance(i, (int, z3.IntNumRef)):
            return traverse_z3(j, hm)
        elif isinstance(j, (int, z3.IntNumRef)):
            return traverse_z3(i, hm)
        else:
            if i.is_string() and j.is_string():
                if i.is_string_value():
                    hm[i.as_string()[1:-1]] = unwrap_substrings(j)
                elif j.is_string_value():
                    hm[j.as_string()[1:-1]] = unwrap_substrings(i)
            else:
                assert False  # for now
    elif n == '<' or n == '>':
        i, j = p.children()
        if isinstance(i, (int, z3.IntNumRef)):
            return traverse_z3(j, hm)
        elif isinstance(j, (int, z3.IntNumRef)):
            return traverse_z3(i, hm)
        else:
            assert False
    return p

In [ ]:
comparisons = {}
for p in _.path:
    traverse_z3(p, comparisons)
comparisons

All that we need now is to declare string variables that match the substrings in `comparisons`, and solve for them for each item in the path. For that, we define `find_alternatives()`.

In [ ]:
def find_alternatives(spans, cmp):
    alts = {}
    for key in spans:
        start, l = spans[key]
        rset = set(range(start, start + l))
        for ckey in cmp:
            cstart, cl = cmp[ckey]
            cset = set(range(cstart, cstart + cl))
            # if rset.issubset(cset): <- ignoring subsets for now.
            if rset == cset:
                if key not in alts:
                    alts[key] = set()
                alts[key].add(ckey)
    return alts

We use it as follows.

In [ ]:
alternatives = find_alternatives(span_hm, comparisons)
alternatives

So, we have our alternatives for each key in the grammar. We can now update our grammar as follows.

In [ ]:
INVENTORY_GRAMMAR_NEW = dict(INVENTORY_GRAMMAR)

In [ ]:
for k in alternatives:
    INVENTORY_GRAMMAR_NEW[k] = INVENTORY_GRAMMAR_NEW[k] + list(alternatives[k])

We made a choice here. We could have completely overwritten the definition of `<table>` . Instead, we added our new alternatives to the existing definition. This way, our fuzzer will also attempt other values for `<table>` once in a while.

In [ ]:
INVENTORY_GRAMMAR_NEW['<table>']

Let us try fuzzing with our new grammar.

In [ ]:
gf = CGrammarFuzzer(INVENTORY_GRAMMAR_NEW)
qtree = gf.fuzz_tree()
query = ostr(gf.tree_to_string(qtree))
try:
    print(repr(query))
    with ConcolicTracer() as _:
        res = _[db_select](str(query))
    print(repr(res))
    print(_.pred(True, True))
except SQLException as e:
    print(e)
except:
    traceback.print_exc()

That is, we were able to reach the dangerous method `my_eval()`.
In effect, what we have done is to lift parts of predicates to the grammar. The new grammar can generate inputs that reach deeper into the program than before. Note that we have only handled the equality predicate. One can also lift the '<' and '>' comparison operators to the grammar if required.

### Limitations

As with dynamic taint analysis, implicit control flow can obscure the predicates encountered during concolic execution. Similarly, calls to internal C functions can cause the symbolic information to be discarded, and only partial information may be obtained.

## Lessons Learned

* String-based and character-based taints allow to dynamically track the information flow form input to the internals of a system and back to the output.

* Checking taints allows to discover untrusted inputs and information leakage at runtime.

* Data conversions and implicit data flow may strip taint information; the resulting untainted strings should be treated as having the worst possible taint.

* Taints can be used in conjunction with fuzzing to provide a more robust indication of incorrect behavior than to simply rely on program crashes.

* Concolic execution can often provide more information than taint analysis with respect to the program behavior. However, this comes at a much larger runtime cost. Hence, unlike taint analysis, real-time analysis is often not possible.

* Similar to taint analysis, concolic execution also suffers from limitations such as indirect control flow and internal function calls.

* Predicates from concolic execution can be used in conjunction with fuzzing to provide an even more robust indication of incorrect behavior than taints, and can be used to create grammars that are better at producing valid inputs.

## Next Steps

A costlier but stronger alternative to concolic fuzzing is [symbolic fuzzing](SymbolicFuzzer.ipynb). Similarly, [search based fuzzing](SearchBasedFuzzer.ipynb) can often provide a cheaper exploration strategy than relying on SMT solvers to provide inputs slightly different from the current path. The chapter on [oracles](Oracles.ipynb) discusses how to provide better oracles in detail.

## Background

Taint analysis on Python using a library approach as we implemented in this chapter was discussed by Conti et al. \cite{Conti2010}. The concolic execution technique used in this chapter was pioneered by PeerCheck \cite{PeerCheck}, and Python Error Finder \cite{Barsotti2018}.

## Exercises

For example, say we want to compute the binomial coefficient by the following formulas

$$ 
^nP_k=\frac{n!}{(n-k)!}
$$

$$
\binom nk=\,^nC_k=\frac{^nP_k}{k!}
$$


In [ ]:
def factorial(n):
    v = 1
    while n != 0:
        v = v * n
        n -= 1
    return v


def permutation(n, k):
    return factorial(n) / factorial(n - k)


def combination(n, k):
    return permutation(n, k) / factorial(k)


def binomial(n, k):
    if n < 0 or k < 0 or n < k:
        raise Exception('Invalid values')
    return combination(n, k)

In [ ]:
binomial(4, 2)